In [7]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf
from tensorflow.python.ops import math_ops as tfmath_ops
from utils import Make_Video_batch, make_checkpoint_folder
from utils import build_video_batch_graph, plot_latents, MSE_rotation
from utils import pandas_res_saver
import sys
import time
import pickle
import os
from utils_circles_grid import Make_circles, Make_squares, plot_circle
from utils_circles_grid import plot_heatmap, plot_square
from GPVAEmodel import *

In [2]:
default_base_dir = os.getcwd()

parser = argparse.ArgumentParser(description='Train GPP-VAE')
parser.add_argument('--steps', type=int, default=50000, help='Number of steps of Adam')
parser.add_argument('--beta0', type=float, default=1, help='initial beta annealing value')
parser.add_argument('--elbo', type=str, choices=['SIN', 'NP'], default='SIN',
                     help='Structured Inf Nets ELBO or Neural Processes ELBO')
parser.add_argument('--modellt', type=float, default=5, help='time scale of model to fit to data')
parser.add_argument('--base_dir', type=str, default=default_base_dir, help='folder within a new dir is made for each run')
parser.add_argument('--expid', type=str, default="debug", help='give this experiment a name')
parser.add_argument('--ram', type=float, default=0.5, help='fraction of GPU ram to use')
parser.add_argument('--seed', type=int, default=None, help='seed for rng')

args = parser.parse_args([])

In [12]:
batch = 35
tmax = 30
px = 32
py = 32
r = 3
vid_lt = 5
model_lt = args.modellt
# batch = 35
# tmax = 12
# px = 12
# py = 12
# pz = 12
# r = 2
# vid_lt = 5
# model_lt = args.modellt

In [15]:
beta = tf.compat.v1.placeholder(dtype=tf.float32, shape=())
vid_batch = build_video_batch_graph(batch=batch, tmax=tmax, px=px, py=py, r=r, lt=vid_lt)
s_elbo, s_rec, s_pkl, np_elbo, np_rec, np_pkl, \
    p_m,p_v,q_m,q_v,pred_vid, _ = build_sin_and_np_elbo_graphs(vid_batch, beta, lt=model_lt)

ValueError: Dimensions must be equal, but are 2 and 3 for 'mul_183' (op: 'Mul') with input shapes: [35,30,2], [35,30,3].